In [1]:
import os
import sys

sys.path.append('..')
from models import *
import numpy as np
import pandas as pd
import matplotlib as plt
import sklearn
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from numpy import zeros
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn import datasets
from sklearn.externals import joblib
import random

le = LabelEncoder()

df_data = pd.read_hdf("../data_extract/features_dataset_2b.h5")

In [2]:
ids = [i for i in range(40)]
for i in [5,31,35,39]:
    ids.remove(i)
random.shuffle(ids)

NUM_PEOPLE = 8
sel = [ids[i] for i in range(len(ids)) if i<NUM_PEOPLE]

bool_arr = np.array([(row in sel) for row in df_data['id']])
df = df_data[bool_arr]

In [3]:
clist = ["chroma{}".format(i) for i in range(12)]
lclist = ["lc{}".format(i) for i in range(12)]
df[lclist] = np.log(df[clist])
flist = ["mfcc{}".format(i) for i in range(14)]+lclist+["centroid","crest","flatness","kurtosis","mean"]
df_train, df_cross, df_test = get_partitions_2(df,train_per=0.3,cross_per=0.6,test_per=0.1)

/home/anshul/anaconda2/envs/voice/lib/python2.7/site-packages/pandas/core/frame.py:2440: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [4]:
obj = SVM(flist)
obj.fit(df_train)
#joblib.dump(obj, '../saved_models/SVM5.pkl') 

In [97]:
#obj2 = joblib.load('../saved_models/SVM5.pkl') 
output = obj.predict_df(df_test)
print output

             precision    recall  f1-score   support

          6       1.00      1.00      1.00         5
          9       1.00      1.00      1.00         5
         11       1.00      1.00      1.00         5
         13       1.00      1.00      1.00         7
         18       1.00      1.00      1.00         4
         26       1.00      1.00      1.00         8
         32       1.00      1.00      1.00         7
         33       1.00      1.00      1.00         7

avg / total       1.00      1.00      1.00        48



In [ ]:
orig, pred = obj.get_results(df_test)
conf_arr = metrics.confusion_matrix(orig['pid'], pred['pid'])